In [9]:
# !/usr/bin/env python3
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import cross_decomposition
import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib.font_manager import FontProperties
def getChineseFont():  
    return FontProperties(fname='/System/Library/Fonts/PingFang.ttc',size=15)

In [10]:
df_csv = pd.read_csv('./data/flow_train.csv')
df_csv['date_dt'] = df_csv['date_dt'].apply(lambda x: pd.to_datetime(str(x))) 

In [11]:
df_dist = df_csv.groupby(['district_code'])

In [12]:
def getModel(df_dist_a):
    df_dist_a['S_3_d'] = df_dist_a['dwell'].shift(1).rolling(window=3).mean()
    df_dist_a['S_6_d'] = df_dist_a['dwell'].shift(1).rolling(window=6).mean()
    df_dist_a['S_9_d'] = df_dist_a['dwell'].shift(1).rolling(window=9).mean()
    df_dist_a['S_3_i'] = df_dist_a['flow_in'].shift(1).rolling(window=3).mean()
    df_dist_a['S_6_i'] = df_dist_a['flow_in'].shift(1).rolling(window=6).mean()
    df_dist_a['S_9_i'] = df_dist_a['flow_in'].shift(1).rolling(window=9).mean()
    df_dist_a['S_3_o'] = df_dist_a['flow_out'].shift(1).rolling(window=3).mean()
    df_dist_a['S_6_o'] = df_dist_a['flow_out'].shift(1).rolling(window=6).mean()
    df_dist_a['S_9_o'] = df_dist_a['flow_out'].shift(1).rolling(window=9).mean()
    df_dist_a_new = df_dist_a.dropna()
    X = df_dist_a_new[['S_3_d','S_6_d','S_9_d', 'S_3_i','S_6_i','S_9_i', 'S_3_o','S_6_o','S_9_o']]
    Y = df_dist_a_new[['dwell', 'flow_in', 'flow_out']]
    t=.8   
    t = int(t*len(df_dist_a_new))   
    # Train dataset   
    X_train = X[:t]   
    Y_train = Y[:t]    
    # Test dataset   
    X_test = X[t:]   
    Y_test = Y[t:] 
    X_train = X[:-1]   
    Y_train = Y[:-1]
    linear = linear_model.LinearRegression().fit(X_train,Y_train)  
    return linear, X_test, Y_test

In [13]:
df_dist_a = list(df_dist)[0][1]
model, x_test, y_test = getModel(df_dist_a.copy())
predicted_price = model.predict(x_test)
# getModel(df_dist_a)

In [14]:
def predict_by_day(x_in, linear):
#     print('x_in length: {}'.format(len(x_in)))
    x_in['S_3_d'] = x_in['dwell'].shift(1).rolling(window=3).mean()
    x_in['S_6_d'] = x_in['dwell'].shift(1).rolling(window=6).mean()
    x_in['S_9_d'] = x_in['dwell'].shift(1).rolling(window=9).mean()
    x_in['S_3_i'] = x_in['flow_in'].shift(1).rolling(window=3).mean()
    x_in['S_6_i'] = x_in['flow_in'].shift(1).rolling(window=6).mean()
    x_in['S_9_i'] = x_in['flow_in'].shift(1).rolling(window=9).mean()
    x_in['S_3_o'] = x_in['flow_out'].shift(1).rolling(window=3).mean()
    x_in['S_6_o'] = x_in['flow_out'].shift(1).rolling(window=6).mean()
    x_in['S_9_o'] = x_in['flow_out'].shift(1).rolling(window=9).mean()
#     print('x_in length: {}'.format(len(x_in)))
#     print(x_in.tail(10))
    x_in_new = x_in.dropna()
#     print('x_in_new length: {}'.format(len(x_in_new)))
    x_ori = x_in_new[['S_3_d','S_6_d','S_9_d', 'S_3_i','S_6_i','S_9_i', 'S_3_o','S_6_o','S_9_o']]
#     print('x_ori length: {}'.format(len(x_ori)))
#     print(x_ori.tail(10))
    y_predict = linear.predict(x_ori)
    return y_predict[-1]

In [15]:
def predict_by_dist(df_dist_ac, linear):
    district_code = df_dist_ac['district_code'].iloc[0]
    city_code = df_dist_ac['city_code'].iloc[0]
    rng = pd.date_range('7/1/2018', periods=15, freq='D')
    for i in range(15):
#         print(len(df_dist_ac))
        y_predict = predict_by_day(df_dist_ac, linear)
#         print('y_predict: {}'.format(y_predict))
        df_dist_ac = df_dist_ac[['date_dt', 'dwell', 'flow_in', 'flow_out', 'city_code', 'district_code']]
        df_dist_ac = df_dist_ac.append({'dwell':y_predict[0], 'flow_in':y_predict[1], 'flow_out':y_predict[2], 'date_dt':rng[i], 'city_code': city_code, 'district_code': district_code}, ignore_index=True)
    return df_dist_ac[-15:]

In [16]:
df_dist_ac = df_dist_a.copy()
# df_dist_ac['district_code'].head(5)
predict_dist = predict_by_dist(df_dist_ac, model)
predict_dist.head(5)
# predict_by_dist(df_dist_ac, model)

,date_dt,dwell,flow_in,flow_out,city_code,district_code
274,2018-07-01,95.837692,76.709246,76.116582,3f7f0ce35d6d0a08377eb2efe2189f4f,01bb949d16b241654d3be95fa224abc8
275,2018-07-02,95.814593,77.596579,77.507622,3f7f0ce35d6d0a08377eb2efe2189f4f,01bb949d16b241654d3be95fa224abc8
276,2018-07-03,96.744424,76.358580,77.276658,3f7f0ce35d6d0a08377eb2efe2189f4f,01bb949d16b241654d3be95fa224abc8
277,2018-07-04,97.242591,76.735391,77.687176,3f7f0ce35d6d0a08377eb2efe2189f4f,01bb949d16b241654d3be95fa224abc8
278,2018-07-05,97.087881,75.837916,76.995722,3f7f0ce35d6d0a08377eb2efe2189f4f,01bb949d16b241654d3be95fa224abc8


In [18]:
def predict(df_disc):
    df_dist_list = list(df_dist)
    predict_result = None
    for i in range(len(df_dist_list)):
        df_dist_a = df_dist_list[i][1]
        model, x_test, y_test = getModel(df_dist_a.copy())
        df_dist_ac = df_dist_a.copy()
        predict_dist = predict_by_dist(df_dist_ac, model)
        if i == 0:
            predict_result = predict_dist
        else:
            predict_result = pd.concat([predict_result, predict_dist])
    return predict_result

In [19]:
predict_result = predict(df_dist)
predict_result.head(5)

,date_dt,dwell,flow_in,flow_out,city_code,district_code
274,2018-07-01,95.837692,76.709246,76.116582,3f7f0ce35d6d0a08377eb2efe2189f4f,01bb949d16b241654d3be95fa224abc8
275,2018-07-02,95.814593,77.596579,77.507622,3f7f0ce35d6d0a08377eb2efe2189f4f,01bb949d16b241654d3be95fa224abc8
276,2018-07-03,96.744424,76.358580,77.276658,3f7f0ce35d6d0a08377eb2efe2189f4f,01bb949d16b241654d3be95fa224abc8
277,2018-07-04,97.242591,76.735391,77.687176,3f7f0ce35d6d0a08377eb2efe2189f4f,01bb949d16b241654d3be95fa224abc8
278,2018-07-05,97.087881,75.837916,76.995722,3f7f0ce35d6d0a08377eb2efe2189f4f,01bb949d16b241654d3be95fa224abc8


In [20]:
len(predict_result)

1470

In [21]:
predict_result['date_dt'] = predict_result['date_dt'].apply(lambda x: int(x.strftime('%Y%m%d')))

In [22]:
predict_result.head(5)

,date_dt,dwell,flow_in,flow_out,city_code,district_code
274,20180701,95.837692,76.709246,76.116582,3f7f0ce35d6d0a08377eb2efe2189f4f,01bb949d16b241654d3be95fa224abc8
275,20180702,95.814593,77.596579,77.507622,3f7f0ce35d6d0a08377eb2efe2189f4f,01bb949d16b241654d3be95fa224abc8
276,20180703,96.744424,76.358580,77.276658,3f7f0ce35d6d0a08377eb2efe2189f4f,01bb949d16b241654d3be95fa224abc8
277,20180704,97.242591,76.735391,77.687176,3f7f0ce35d6d0a08377eb2efe2189f4f,01bb949d16b241654d3be95fa224abc8
278,20180705,97.087881,75.837916,76.995722,3f7f0ce35d6d0a08377eb2efe2189f4f,01bb949d16b241654d3be95fa224abc8


In [26]:
predict_result = predict_result[['date_dt','city_code', 'district_code', 'dwell', 'flow_in', 'flow_out']]
predict_result.to_csv('./data/prediction.csv', header=False, index=False)